<a href="https://colab.research.google.com/github/aszigeti18/skills-copilot-codespaces-vscode/blob/main/Momentum_Breakout_Scanner_Finnhub_2025_05_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📈 Finnhub Momentum Breakout Scanner
This notebook scans tickers for breakout patterns using historical daily data from the Finnhub API. Replace `YOUR_API_KEY` with your Finnhub API key.


In [1]:
# 🚩 Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 📦 Install necessary packages
!pip install requests pandas matplotlib --quiet

In [3]:
# 📚 Import libraries
import requests
import pandas as pd
import numpy as np
import datetime
import time
import os
import matplotlib.pyplot as plt

In [4]:
# 🔑 Set Finnhub API key
API_KEY = 'd0e4r4hr01qv1dml1th0d0e4r4hr01qv1dml1thg'  # Replace this with your actual Finnhub API key

In [5]:
# 📄 Load ticker symbols from file
with open('/content/drive/My Drive/Data/Tickers.txt', 'r') as file:
    tickers = [line.strip().upper() for line in file.readlines()]

In [6]:
# 📊 Function to get historical daily data
def get_stock_data(symbol):
    end = int(time.time())
    start = end - (300 * 86400)
    url = f"https://finnhub.io/api/v1/stock/candle?symbol={symbol}&resolution=D&from={start}&to={end}&token={API_KEY}"
    response = requests.get(url)
    data = response.json()

    if data.get('s') != 'ok':  # Check for error status
        print(f"Error fetching data for {symbol}: {data.get('s')}")  # Log error messages
        return pd.DataFrame()  # Return empty DataFrame if error

    df = pd.DataFrame(data)
    df['t'] = pd.to_datetime(df['t'], unit='s')
    df.set_index('t', inplace=True)
    df.rename(columns={'c': 'Close', 'h': 'High', 'l': 'Low', 'o': 'Open', 'v': 'Volume'}, inplace=True)
    return df[['Open', 'High', 'Low', 'Close', 'Volume']]

In [7]:
# 📈 Analyze each ticker for breakout
def analyze_stock(ticker):
    df = get_stock_data(ticker)
    if df.empty or df.shape[0] < 205:
        return None

    df['20High'] = df['High'].rolling(20).max()
    df['50SMA'] = df['Close'].rolling(50).mean()
    df['200SMA'] = df['Close'].rolling(200).mean()
    df['20VolumeAvg'] = df['Volume'].rolling(20).mean()

    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    latest = df.iloc[-1]
    breakout = (
        latest['Close'] > latest['20High'] and
        latest['Volume'] > 1.5 * latest['20VolumeAvg'] and
        latest['RSI'] > 60 and
        latest['Close'] > latest['50SMA'] and
        latest['Close'] > latest['200SMA']
    )

    if breakout:
        close_price = latest['Close']
        follow_through = {}
        for i in range(1, 6):
            try:
                future_close = df.iloc[-1 + i]['Close']
                pct_change = ((future_close - close_price) / close_price) * 100
                follow_through[f'Day+{i} %'] = round(pct_change, 2)
            except IndexError:
                follow_through[f'Day+{i} %'] = 'N/A'

        return {
            'Ticker': ticker,
            'Close': round(close_price, 2),
            '20DayHigh': round(latest['20High'], 2),
            'Volume': int(latest['Volume']),
            'RSI': round(latest['RSI'], 2),
            **follow_through
        }

    return None

In [8]:
# ▶️ Run scan
results = []
for ticker in tickers:
    try:
        result = analyze_stock(ticker)
        if result:
            results.append(result)
            print(f"✅ Breakout Opportunity Found: {result['Ticker']}")
    except Exception as e:
        print(f"⚠️ Error: {ticker} — {e}")
    time.sleep(1.2)  # Avoid hitting 60 calls/min limit

Error fetching data for SYMBOL: None
Error fetching data for AAPL: None
Error fetching data for NVDA: None
Error fetching data for MSFT: None
Error fetching data for AMZN: None
Error fetching data for META: None
Error fetching data for GOOGL: None
Error fetching data for TSLA: None
Error fetching data for AVGO: None
Error fetching data for GOOG: None
Error fetching data for BRK.B: None
Error fetching data for JPM: None
Error fetching data for LLY: None
Error fetching data for V: None
Error fetching data for XOM: None
Error fetching data for UNH: None
Error fetching data for MA: None
Error fetching data for COST: None
Error fetching data for WMT: None
Error fetching data for HD: None
Error fetching data for PG: None
Error fetching data for NFLX: None
Error fetching data for JNJ: None
Error fetching data for ABBV: None
Error fetching data for CRM: None
Error fetching data for BAC: None
Error fetching data for ORCL: None
Error fetching data for MRK: None
Error fetching data for CVX: None


KeyboardInterrupt: 

In [ ]:
# 💾 Save results
df_results = pd.DataFrame(results)
print("\n📊 Final Breakout Candidates:")
print(df_results)

if not df_results.empty:
    today_str = datetime.datetime.today().strftime('%Y_%m_%d')
    output_dir = "/content/drive/My Drive/Stocks"
    os.makedirs(output_dir, exist_ok=True)
    df_results.to_csv(f"{output_dir}/Momentum_Finnhub_{today_str}.txt", sep='\t', index=False)
    print(f"\n📁 Results saved to: {output_dir}/Momentum_Finnhub_{today_str}.txt")